In [1]:
!pip install torch
!pip install torchsummary
!pip install rasterio
!pip install torchmetrics

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

import os
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import rasterio as rio

import torchmetrics as tm

import torchvision.transforms as transforms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 52.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 8.7 MB/s eta 0:00:00


In [2]:
FOLDER = '/content/drive/MyDrive/GIS_ML/EuroSATallBands'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
def get_dimensions(input_dimensions, block_number):
  x = input_dimensions
  for i in range(1,block_number+1,1):
    x = x/2
  return int(x)

In [5]:
print(get_dimensions(128,4))

8


In [6]:
class CNN(nn.Module):
  def __init__(self, multiclasses, input_features, output_features, hidden_layers, max_pooled_dimension):
    super().__init__()
    self.multiclasses = multiclasses
    self.input_features = input_features
    self.output_features = output_features
    self.hidden_layers = hidden_layers
    self.max_pooled_dimension = max_pooled_dimension

    self.cnn_layers = nn.Sequential(
        nn.Conv2d(in_channels=input_features, out_channels=output_features[0], kernel_size=(3,3), padding=1),
        nn.BatchNorm2d(output_features[0]),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),
        nn.Conv2d(in_channels=output_features[0], out_channels=output_features[1], kernel_size=(3,3), padding=1),
        nn.BatchNorm2d(output_features[1]),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),
        nn.Conv2d(in_channels=output_features[1], out_channels=output_features[2], kernel_size=(3,3), padding=1),
        nn.BatchNorm2d(output_features[2]),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),
        nn.Conv2d(in_channels=output_features[2], out_channels=output_features[3], kernel_size=(3,3), padding=1),
        nn.BatchNorm2d(output_features[3]),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2)
    )

    self.fcn_layers = nn.Sequential(
        nn.Linear(max_pooled_dimension*max_pooled_dimension*output_features[3], hidden_layers[0]),
        nn.BatchNorm1d(hidden_layers[0]),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_layers[0], hidden_layers[1]),
        nn.BatchNorm1d(hidden_layers[1]),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_layers[1], multiclasses)
    )

  def forward(self,x):
    x = self.cnn_layers(x)
    x = x.view(-1, self.max_pooled_dimension*self.max_pooled_dimension*self.output_features[3])
    x = self.fcn_layers(x)
    return x

In [7]:
model = CNN(multiclasses=10,
            input_features=10,
            output_features=[10,20,30,40],
            hidden_layers=[268,512],
            max_pooled_dimension=get_dimensions(64,4)).to(device)

In [8]:
model

CNN(
  (cnn_layers): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(30, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=Tru

In [9]:
train = pd.read_csv(FOLDER+"/train.csv")
test = pd.read_csv(FOLDER+"/test.csv")
val = pd.read_csv(FOLDER+"/validation.csv")

# reduction of the data to perform faster calculations
train = train.sample(frac=0.1,random_state=101)
test = test.sample(frac=0.1,random_state=101)
val = val.sample(frac=0.1,random_state=101)

In [10]:
class EuroSatDataSet(Dataset):

    def __init__(self, df):
        super().__init__
        self.df = df

    def __getitem__(self, index):
        image_name = FOLDER + '/' + self.df.iloc[index, 0]
        label = self.df.iloc[index, 1]
        label = np.array(label)
        source = rio.open(image_name)
        image = source.read()
        source.close()
        image = image.astype('float32')
        image = image[[1,2,3,4,5,6,7,8,11,12], :, :]
        image = torch.from_numpy(image)
        label = torch.from_numpy(label)
        label = label.long()
        return image, label

    def __len__(self):
        return len(self.df)

In [11]:
train_dataset = EuroSatDataSet(train)
print(len(train_dataset))

1932


In [12]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, sampler=None,
num_workers=0, pin_memory=False, drop_last=False)

In [13]:
#https://www.binarystudy.com/2021/04/how-to-calculate-mean-standard-deviation-images-pytorch.html - function taken from this input
def batch_mean_and_sd(loader, input_bands):

    count = 0
    fst_moment = torch.empty(input_bands)
    snd_moment = torch.empty(input_bands)

    for images, _ in loader:
        b, c, h, w = images.shape
        pixels = b * h * w
        sum_ = torch.sum(images, dim=[0, 2, 3])
        sum_of_square = torch.sum(images ** 2,
                                  dim=[0, 2, 3])
        fst_moment = (count * fst_moment + sum_) / (count + pixels)
        snd_moment = (count * snd_moment + sum_of_square) / (count + pixels)
        count += pixels

    mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)
    return mean,std

In [ ]:
band_stats = batch_mean_and_sd(train_dataloader, 10)

In [ ]:
class EuroSat(Dataset):

    def __init__(self, df, image_mean, image_std, transform):
        self.df = df
        self.image_mean = image_mean
        self.image_std = image_std
        self.transform = transform

    def __getitem__(self, index):
        image_name = FOLDER + '/' + self.df.iloc[index, 0]
        label = self.df.iloc[index, 1]
        label = np.array(label)
        source = rio.open(image_name)
        image = source.read()
        source.close()
        image = image[[1,2,3,4,5,6,7,8,11,12], :, :]
        image = np.subtract(image, self.image_mean)
        image = np.divide(image, self.image_std)
        image = image.astype('float32')
        image = torch.from_numpy(image)
        label = torch.from_numpy(label)
        label = label.long()
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.df)

In [ ]:
band_means = np.array(band_stats[0].tolist())
band_stds = np.array(band_stats[1].tolist())

In [ ]:
image_mean = np.repeat(band_means[0], 64*64).reshape((64,64,1))
for b in range(1,len(band_means)):
    image_mean_2 = np.repeat(band_means[b], 64*64).reshape((64,64,1))
    image_mean = np.dstack([image_mean, image_mean_2])
image_mean = np.transpose(image_mean, (2,0,1))

image_std = np.repeat(band_stds[0], 64*64).reshape((64,64,1))
for b in range(1,len(band_stds)):
    image_std_2 = np.repeat(band_stds[b], 64*64).reshape((64,64,1))
    image_std = np.dstack([image_std, image_std_2])
image_std = np.transpose(image_std, (2,0,1))

In [ ]:
simple_transformation = transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),]
    )

In [ ]:
train_dataset = EuroSat(train, image_mean, image_std, transform=simple_transformation)
validation_dataset = EuroSat(val, image_mean, image_std, transform=None)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, sampler=None,
num_workers=0, pin_memory=False, drop_last=True)

validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=False, sampler=None,
num_workers=0, pin_memory=False, drop_last=True)

In [ ]:
batch = next(iter(train_dataloader))
images, labels = batch
print(f'Batch Image Shape: {images.shape}, Batch Label Shape: {labels.shape}')
print(f'Batch Image Data Type: {images.dtype}, Batch Label Data Type: {labels.dtype}')
print(f'Batch Image Band Means: {torch.mean(images, dim=(0,2,3))}')
print(f'Batch Label Minimum: {torch.min(labels, dim=0)}, Batch Label Maximum: {torch.max(labels, dim=0)}')

In [ ]:
test_image = images[1]
test_mask = labels[1]
print(f'Image Shape: {test_image.shape}, Label Shape: {test_mask.shape}')
print(f'Image Data Type: {test_image.dtype}, Label Data Type: {test_mask.dtype}')

In [ ]:
def img_display(img, image_mean, image_std):
    img = np.multiply(img, image_std)
    img = np.add(img, image_mean)
    image_vis = img[[2,1,0],:,:]
    image_vis = image_vis.permute(1,2,0)
    image_vis = (image_vis.numpy()/4000)*255
    image_vis = image_vis.astype('uint8')
    return image_vis

dataiter = iter(train_dataloader)
images, labels = next(dataiter)
cover_types = {0: 'Annual Crop',
1: 'Forest',
2: 'Herb Veg',
3: 'Highway',
4: 'Industrial',
5: 'Pasture',
6: 'Perm Crop',
7: 'Residential',
8: 'River',
9: 'SeaLake'}

In [ ]:
fig, axis = plt.subplots(4, 8, figsize=(15, 10))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        image, label = images[i], labels[i]
        ax.imshow(img_display(image, image_mean, image_std)) # add image
        ax.set(title = f"{cover_types[label.item()]}") # add label
        ax.axis('off')

In [ ]:
optimizer = torch.optim.AdamW(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
acc = tm.Accuracy(task="multiclass", num_classes=10).to(device)
f1 = tm.F1Score(task="multiclass", num_classes=10).to(device)
kappa = tm.CohenKappa(task="multiclass", num_classes=10).to(device)

In [ ]:
epochs = 50
saveFolder = ''

In [ ]:
epoch_number = []
train_loss = []
train_acc = []
train_f1 = []
train_kappa = []
validation_loss = []
validation_acc = []
validation_f1 = []
validation_kappa = []

f1VMax = 0.0

# Loop over epochs
for epoch in range(1, epochs+1):
    # Loop over training batches
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        # Get data and move to device
        inputs, targets = inputs.to(device), targets.to(device)

        # Clear gradients
        optimizer.zero_grad()
        # Predict data
        outputs = model(inputs)
        # Calculate loss
        loss = criterion(outputs, targets)

        # Calculate metrics
        accT = acc(outputs, targets)
        f1T = f1(outputs, targets)
        kappaT = kappa(outputs, targets)

        # Backpropagate
        loss.backward()

        # Update parameters
        optimizer.step()

    # Accumulate metrics at end of training epoch
    accT = acc.compute()
    f1T = f1.compute()
    kappaT = kappa.compute()

    # Print Losses and metrics at end of each training epoch
    print(f'Epoch: {epoch}, Training Loss: {loss.item():.4f}, Training Accuracy: {accT:.4f}, Training F1: {f1T:.4f}, Training Kappa: {kappaT:.4f}')

    # Append results
    epoch_number.append(epoch)
    train_loss.append(loss.item())
    train_acc.append(accT.detach().cpu().numpy())
    train_f1.append(f1T.detach().cpu().numpy())
    train_kappa.append(kappaT.detach().cpu().numpy())

    # Reset metrics
    acc.reset()
    f1.reset()
    kappa.reset()

    # loop over validation batches
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(validation_dataloader):
            # Get data and move to device
            inputs, targets = inputs.to(device), targets.to(device)

            # Predict data
            outputs = model(inputs)
            # Calculate validation loss
            loss_v = criterion(outputs, targets)

            # Calculate metrics
            accV = acc(outputs, targets)
            f1V = f1(outputs, targets)
            kappaV = kappa(outputs, targets)

    # Accumulate metrics at end of validation epoch
    accV = acc.compute()
    f1V = f1.compute()
    kappaV = kappa.compute()

    # Print validation loss and metrics
    print(f'Validation Loss: {loss_v.item():.4f}, Validation Accuracy: {accV:.4f}, Validation F1: {f1V:.4f}, Validation Kappa: {kappaV:.4f}')

    # Append results
    validation_loss.append(loss_v.item())
    validation_acc.append(accV.detach().cpu().numpy())
    validation_f1.append(f1V.detach().cpu().numpy())
    validation_kappa.append(kappaV.detach().cpu().numpy())

    # Reset metrics
    acc.reset()
    f1.reset()
    kappa.reset()

    # Save model if validation F1-score improves
    f1V2 = f1V.detach().cpu().numpy()
    if f1V2 > f1VMax:
        f1VMax = f1V2
        torch.save(model.state_dict(), saveFolder + 'eurosat_model.pt')
        print(f'Model saved for epoch {epoch}.')

In [ ]:
SeNum = pd.Series(epoch_number, name="epoch")
St_loss = pd.Series(train_loss, name="training_loss")
St_acc = pd.Series(train_acc, name="training_accuracy")
St_f1 = pd.Series(train_f1, name="training_f1")
St_kappa = pd.Series(train_kappa, name="training_kappa")
Sv_loss = pd.Series(validation_loss, name="val_loss")
Sv_acc = pd.Series(validation_acc, name="val_accuracy")
Sv_f1 = pd.Series(validation_f1, name="val_f1")
Sv_kappa = pd.Series(train_kappa, name="val_kappa")
results_df = pd.concat([SeNum, St_loss, St_acc, St_f1, St_kappa, Sv_loss, Sv_acc, Sv_f1, Sv_kappa], axis=1)

In [ ]:
results_df.to_csv(saveFolder+"resultsCNN.csv")

In [ ]:
results_df = pd.read_csv(saveFolder+"resultsCNN.csv")

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
first_plot = results_df.plot(x='epoch', y="training_loss")
results_df.plot(x='epoch', y="val_loss", ax=first_plot)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
first_plot = results_df.plot(x="epoch", y="training_f1")
results_df.plot(x='epoch', y="val_f1", ax=first_plot)
plt.show()